## Import Necessary Modules 

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
import ipywidgets as widgets
from IPython.display import display, HTML
from langchain_core.documents import Document
from langchain_chroma import Chroma
import os
import json
import requests
from langchain_community.vectorstores import Chroma
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain_chroma import Chroma
from langchain.agents import create_tool_calling_agent
from datetime import datetime
import pytz
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate


### Load Environment Variable

In [ ]:
# Load the API key from .env file
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

## Populating the Vector Database
Check populate_vector_bd.ipynb file

## Memory Configuration - Short-Term and Long-Term

In [ ]:
book_persist_directory="./Dataset/books_chroma"
longterm_memory_persist_dir = "./Dataset/langchain_conversation_chroma"
embedder = HuggingFaceEmbeddings(model_name="BAAI/llm-embedder") # Embeddings

In [ ]:
# ================== Memory Configuration ==================
# Short-term memory (last 6 messages)
short_term_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True
)

# Long-term memory using Chroma
class LongTermMemory:
    def __init__(self):
        self.vectorstore = Chroma(
            collection_name="conversation_history",
            persist_directory=longterm_memory_persist_dir,
            embedding_function=embedder
        )
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 2})

    def get_relevant_memories(self, query):
        return self.retriever.invoke(query)

    def add_memory(self, text):
        tz_Mumbai = pytz.timezone('Asia/Kolkata')
        datetime_Mumbai = datetime.now(tz_Mumbai)
        self.vectorstore.add_texts(
            texts=[text],
            metadatas=[{"type": "conversation",
                        "Time Date": str(datetime_Mumbai)}]
        )

long_term_memory = LongTermMemory()


In [ ]:
lt_context = long_term_memory.get_relevant_memories("")
lt_context

In [ ]:
short_term_memory

## Build Agents - Book Analysis and Internet Search

In [ ]:
# ================== Book Analysis Agent ==================
class BookAnalysisAgent:
    def __init__(self):        
        # Create LangChain vectorstore
        self.vectorstore = Chroma(persist_directory=book_persist_directory, 
                collection_name="Adavance_RAG_Test",
                embedding_function=embedder)
        
        self.retriever = self.vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

    def answer_book(self, query):
        """Process book queries with error handling"""
        try:
            docs = self.retriever.invoke(query)
            if not docs:
                return "No relevant book passages found."
            return "\n\n".join([f"From {doc.metadata['title']}:\n{doc.page_content}" for doc in docs])
            # print("\n")
            # print("docs: ", str(docs))
            # return docs
        except Exception as e:
            return f"Book search error: {str(e)}"

# ================== Internet Search Agent ==================
class InternetSearchAgent:
    def __init__(self):
        self.url = "https://google.serper.dev/search"
        self.headers = {
        'X-API-KEY': SERP_API_KEY,
        'Content-Type': 'application/json'
        }
        self.history = []

    def search_web(self, query):
        """Perform web search with error handling"""
        try:
            payload = json.dumps({
                "q": query,
                "location": "India",
                "gl": "in"
            })
            response = requests.post(self.url, headers=self.headers, data=payload, timeout=10)
            response.raise_for_status()
            
            result = response.json()
            self.history.append({"query": query, "result": result})
            
            # Extract and format relevant information
            if 'organic' not in result:
                return "No relevant web results found."
                
            top_results = result['organic'][:3]
            return "\n".join([f"{res['title']}: {res.get('snippet', '')}" for res in top_results])
            
        except Exception as e:
            return f"Search error: {str(e)}"


#### Initialize agents

In [ ]:
# Initialize agents
book_agent = BookAnalysisAgent()
internet_agent = InternetSearchAgent()

tools = [
    Tool(
        name="Literary Analysis",
        func=book_agent.answer_book,
        description="Analysis of book themes and content"
    ),
    Tool(
        name="Web Search",
        func=internet_agent.search_web,
        description="Current information and general knowledge"
    )
]


In [ ]:
# ================== Agent Setup ==================
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix="""Provide Detail Answer for questions using context from memories and tools. Follow these rules:
    1. Use Literary Analysis for book-related questions.  
    2. Use Web Search for current events/general knowledge.
    3. Consider both conversation history and long-term memories.
    4. Generate detailed and precise Actiona Input for Literary Analysis and Web Search.
    5. If you not able to get the exact anwser from memories and tools reformulate the query with more details.""",
    suffix="""### Current Conversation:
{chat_history}

{input}

{agent_scratchpad}""",
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

# Initialize LLM (replace with your preferred model)
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",
    api_key=GROQ_API_KEY
)

# # your LCEL-style chain
llm_chain = prompt | llm
# new agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create executor with combined memory handling
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=short_term_memory,
    verbose=False,
    max_iterations=5,
    handle_parsing_errors=True
)

### Enhance Query - Add Long-Term Memory

In [ ]:
# ================== Enhanced Query Handling ==================
# Define a function to extract the content of a message
def process_query(query):
    # Get relevant long-term memories
    lt_context = "\n\n".join([doc.page_content for doc in long_term_memory.get_relevant_memories(query)])
    # print("\n")
    # print("lt_context: ",lt_context)
    input = f"""### Long Term Memories: {lt_context}

### User Question: {query}
"""
    # Execute agent
    result = agent_executor.invoke({"input": input})
    agent_ans = result['output'].split("Final Answer:")[1].strip()
    # Store in long-term memory
    long_term_memory.add_memory(f"User: {query}\nAgent: {agent_ans}")
    
    return result['output']

In [ ]:
# Define the chatbot response function
def chatbot_response(user_input):
    # Finally, let's invoke the chain
    response = process_query(user_input)
    return f"{response}"

# Create the chatbot UI
# Text input for user messages
user_input = widgets.Text(
    placeholder="Type your message here...",
    description="You:",
    layout=widgets.Layout(width="80%")
)

# Button to submit messages
submit_button = widgets.Button(
    description="Send",
    button_style="success"
)

# Output area for the conversation
output = widgets.Output(
    layout=widgets.Layout(),
    style={"description_width": "initial"}
)

# Function to handle button click
def on_submit_button_click(b):
    with output:
        user_message = user_input.value
        if user_message.strip():  # Check if the input is not empty
            # Display the user's message
            display(HTML(f"<strong>You:</strong> {user_message}"))
            
            # Get the chatbot's response
            bot_response = chatbot_response(user_message)

            # Extract the content within the <think> tag
            think_content = bot_response.split('Final Answer:')[0].strip()

            # Extract the bot's response after the <think> tag
            answer_content = bot_response.split('Final Answer:')[1].strip()

            # Format the output
            formatted_output = f"""
            <strong>AskAI AgentExecutor Thinking:</strong><think style="font-family: 'Courier New', Courier, monospace;">
            > Entering new AgentExecutor chain...
            {think_content}
            > Finished chain.
            </think>
            <strong>AskAI Answer:</strong> {answer_content}
            """
            # formatted_output = f"""<strong>AskAI Answer:</strong> {answer_content}"""
            formatted_output_html = formatted_output.replace("\n", "<br>")
            # Display the bot's response
            display(HTML(f"{formatted_output_html}"))
            display(HTML("<br>"))
            # Clear the input box
            user_input.value = ""
        else:
            display(HTML("<em>Please enter a message.</em>"))

# Attach the function to the button's click event
submit_button.on_click(on_submit_button_click)

# Arrange the widgets vertically
chatbot_ui = widgets.VBox([user_input, submit_button, output])

# Display the chatbot UI
display(chatbot_ui)

---
- You: What is the main theme of Verity?
- AskAI Answer: TThe main theme of "Verity" by Colleen Hoover revolves around the complexity of truth, psychological manipulation, and the intricate dynamics of human relationships. The novel explores how individuals can hide their true selves, the impact of past traumas on present behavior, and the moral ambiguities that arise in personal and professional contexts. The title itself, "Verity," meaning truth, underscores the journey of uncovering hidden truths about the characters and their pasts. Additionally, the book delves into themes of self-discovery and the struggle with internal demons, highlighting the darker aspects of human nature and relationships.
---
- You: Who murdered Megan Hipwell?
- AskAI Answer: The murderer of Megan Hipwell is not explicitly identified in the provided documents. However, based on the context and clues, it can be inferred that her husband might be the killer, as he is a potential suspect due to the circumstances surrounding her disappearance and the mention of an affair. Answer: The murderer of Megan Hipwell is not explicitly identified in the provided context, but it is suggested that her husband might be the killer.
---
- You: What last 2 questions did I ask?
- AskAI Answer: The last two questions you asked were: 1. Who murdered Megan Hipwell? 2. What is the main theme of Verity?
---
- You: List all my questions which I asked till now.
- AskAI Answer: Here is a list of all the questions you've asked so far: 1. "What is the main theme of Verity?" 2. "Who murdered Megan Hipwell?" 3. "What last 2 questions did I ask?" 4. "List all my questions which I asked till now."
---
- You: How many childern do Verity and Jeremy have and tell me their children names?
- AskAI Answer: Verity and Jeremy have three children. Their daughters are named Chastin and Harper, and they also have a son named Crew.
---
- You: Who was Crew?
- AskAI Answer: Crew is Jeremy's son, and he plays a significant role in the story. Crew is a young boy who has experienced traumatic events, including witnessing his sister's death and his mother's inaction during the incident. He is close to his father, Jeremy, and is part of a family that includes the narrator, who is pregnant, making Crew a big brother soon. Crew's character deals with adjusting to his new family dynamics and the emotional scars from his past.
---


In [ ]:
# Example query execution
# lt_context:  User: What was my previous question?
# Agent: Your previous question was "What is my name?".

# User: What is my name?
# Agent: Your name is Saurabh.

# User: Hi I am Saurabh
# Agent: Hi Saurabh, how can I assist you today?

# User: I like Pizza
# Agent: Pizza is a delicious food, what's your favorite topping on a pizza?

# User: How many childern do Verity and Jeremy have and tell me their children names?
# Agent: Verity and Jeremy have two daughters, Harper and Crew.


# query = "Verity and Jeremy 3 childern and tell me their children names?"

In [ ]:
# ree = chatbot_response("I like Pizza")

## Testing - Not Important
I you are interested look for knowledge

### long-term memory agent using - VectorStoreRetrieverMemory
Stores the conversation history in a vector store and retrieves the most relevant parts of past conversation based on the input.

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool

In [ ]:
recall_vector_store = InMemoryVectorStore(embedder)

In [ ]:
import uuid


def get_user_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")
    return user_id


@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id = get_user_id(config)
    document = Document(
        page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id}
    )
    recall_vector_store.add_documents([document])
    return memory


@tool
def search_recall_memories(query: str, config: RunnableConfig) -> list[str]:
    """Search for relevant memories."""
    user_id = get_user_id(config)

    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = recall_vector_store.similarity_search(
        query, k=3, filter=_filter_function
    )
    return [document.page_content for document in documents]

In [ ]:
# NOTE: we're specifying `user_id` to save memories for a given user
config = {"configurable": {"user_id": "1", "thread_id": "1"}}
memory = ""
add_memories = search_recall_memories.invoke(memory, config)

In [ ]:

recall_memories = search_recall_memories.invoke(convo_str, config)

### Test

In [ ]:
# # ================== Memory Configuration ==================
# # Short-term memory
# short_term_memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     input_key="input",
#     return_messages=True
# )

# # Long-term memory configuration
# conversation_persist_dir = "./Dataset/conversation_chroma"
# conversation_vectorstore = Chroma(
#     collection_name="conversation_history",
#     persist_directory=conversation_persist_dir,
#     embedding_function=embedder
# )
# conversation_retriever = conversation_vectorstore.as_retriever(search_kwargs={"k": 3})
# long_term_memory = VectorStoreRetrieverMemory(
#     retriever=conversation_retriever,
#     input_key="input"
# )

# # Combined memory system
# memory = CombinedMemory(memories=[short_term_memory, long_term_memory])

# # Initialize agents
# book_agent = BookAnalysisAgent()
# internet_agent = InternetSearchAgent()

# # Create tools
# tools = [
#     Tool(
#         name="Book Search",
#         func=book_agent.answer_book,
#         description="Use for questions about books, authors, or literary content"
#     ),
#     Tool(
#         name="Web Search",
#         func=internet_agent.search_web,
#         description="Use for real-time information or general knowledge questions"
#     )
# ]

# # ================== Agent Setup ==================
# prompt = ZeroShotAgent.create_prompt(
#     tools,
#     prefix="""Answer questions using context from memories and tools. Follow these rules:
#     1. Use Book Search for book-related questions
#     2. Use Web Search for current events/general knowledge
#     3. Consider both conversation history and long-term memories""",
#     suffix="""Long-term Context:
# {history}

# Current Conversation:
# {chat_history}

# Question: {input}
# {agent_scratchpad}""",
#     input_variables=["input", "chat_history", "history", "agent_scratchpad"]
# )

# # Initialize LLM (replace with your preferred model)
# llm = ChatGroq(
#     temperature=0,
#     model_name="llama-3.3-70b-versatile",
#     api_key=GROQ_API_KEY
# )

# # llm_chain = LLMChain(llm=llm, prompt=prompt)
# # agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)

# # # your LCEL-style chain
# llm_chain = prompt | llm
# # new agent
# agent = create_tool_calling_agent(llm, tools, prompt)

# # Create agent executor
# agent_executor = AgentExecutor(
#     agent=agent,
#     tools=tools,
#     memory=memory,
#     verbose=True,
#     max_iterations=3,
#     handle_parsing_errors=True
# )

# Example query execution
# query = "What is the main theme of Verity?"
# result = agent_executor.invoke({"input": query})
